In [11]:
import re
import csv
import time
import nltk
import json
import string
import numpy as np
import pandas as pd
from nltk import stem
import sklearn.metrics
from random import randint
from numpy.linalg import norm
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

with open('../new_userDict.json', 'r') as fp:
    
    new_userDict = json.load(fp)
    
with open('../new_isbnDict.json', 'r') as fp:
    
    new_isbnDict = json.load(fp)
    
with open('../dict_row.json', 'r') as fp:
    
    dict_row = json.load(fp)
    
with open('../dict_col.json', 'r') as fp:
    
    dict_col = json.load(fp)
    
        
print("Ok")

Ok


In [12]:
with open('../clustering/CLUSTERS_ITEMS.json', 'r') as fp:
    
    CLUSTERS_ITEMS = json.load(fp)
    
    
with open('../clustering/CLUSTERS_USERS.json', 'r') as fp:
    
    CLUSTERS_USERS = json.load(fp)
    
    
with open('../clustering/file/clusters_dict_row.json', 'r') as fp:
    
    clusters_dict_row = json.load(fp)
    
    
with open('../clustering/file/clusters_dict_col.json', 'r') as fp:
    
    clusters_dict_col = json.load(fp)
    
    
with open('../clustering/cluster_book_dict.json', 'r') as fp:
    
    cluster_book_dict = json.load(fp)
    
    
with open('../clustering/cluster_users_dict.json', 'r') as fp:
    
    cluster_users_dict = json.load(fp)
    
print("Ok") 



with open('../clustering/index_book_user_clusters/index_user_cluster.json', 'r') as fp:
    
    index_user_cluster = json.load(fp)
    
    
with open('../clustering/index_book_user_clusters/index_book_cluster.json', 'r') as fp:
    
    index_book_cluster = json.load(fp)
    
print("Ok")

Ok
Ok


In [13]:
def booksRatedUser(new_isbnDict, new_userDict, user_number, score):
    
    '''
    
    input:  new_userDict (Dict), user_number (int), score (int)
    
    action: select all books rated by my user
    
    output: books_rated (list)
    
    '''
    
    books_rated = []
    
    for book in new_userDict[str(user_number)]:
        
        if int(new_userDict[str(user_number)][book]) > score:
            
            try:
                new_isbnDict[book]
                books_rated.append(book)
                
            except:
                
                continue
            
    return list(set(books_rated))


def SimilarityBooks(utility_DataFrame, book_number, books_similar):
    
    '''
    
    input: utility_DataFrame (DataFrame), book_number (int), books_similar (List)
    
    action: compute cosine similarity between book_number and all the books in books_similar
    
    output: new_similarity (List of tuples)
    
    '''

    x = utility_DataFrame[str(book_number)]
    x_length = norm(x)
    
    y = utility_DataFrame[books_similar]
    y_length = norm(utility_DataFrame[books_similar],axis=0)

    
    num = (y.T.values*x.values).sum(axis=1)
    den = x_length*y_length

    similarity = num/den
    similarity = np.nan_to_num(similarity)
    
    d = list(zip(list(books_similar),similarity))
    new_similarity = sorted(d, key=lambda tup: tup[1], reverse=True)
    
    return new_similarity


def itemItemsRecommendation(new_similarity, user_number, book_number, k, new_userDict):
    
    
    '''
    
    input:  new_similarity(List of tuples), new_isbnDict(Dict), k(int)
    
    action: compute mean similarity of the first k items
    
    output: recommendation (float)
    
    '''
    
    
    if len(new_similarity) < k:
        
        recommendation = np.mean([u[1] for u in new_similarity])
    
    else:
        
        recommendation = np.mean([u[1] for u in new_similarity[:k]])
        
    
    return recommendation


def itemItemsScore(new_userDict, new_similarity, k, user_number):
    
    score = [int(new_userDict[str(user_number)][u[0]]) for u in new_similarity[:k]]
    
    if score == []:
        
        return None
    
    return np.mean(score)


def CollaborativeFilteringItemItemsRMSE(utility_DataFrame, new_userDict, new_isbnDict, user_number, score_min, book_number, k):
    
    books_rated_user = booksRatedUser(new_isbnDict, new_userDict, user_number,score_min)
    
    if books_rated_user == []:
        
        return None
    
    new_similarity = SimilarityBooks(utility_DataFrame, book_number, books_rated_user)
    
    if new_similarity == []:

        return None
    
    score = itemItemsScore(new_userDict, new_similarity[1:], k, user_number)
    
    return score

In [14]:
def convert(user_number,book_number, index_user_cluster, index_book_cluster):
    
    user_cluster = index_user_cluster[user_number]
    book_cluster = index_book_cluster[book_number]
    
    return user_cluster, book_cluster

In [15]:
def createSampleDict(new_isbnDict, new_userDict, t1, t2):
    

    small_isbnDict = {}

    for book in new_isbnDict:

        temp = new_isbnDict[book]
        i = 0

        for t in temp.values():
            if t != "0":
                i = i+1

        if i >t1:

            small_isbnDict[book] = new_isbnDict[book]   

    small_userDict = {}

    for user in new_userDict:

        temp = new_userDict[user]
        i = 0

        for t in temp.values():
            if t != "0":
                i = i+1

        if i >t2:

            small_userDict[user] = new_userDict[user]
            
            
    return small_isbnDict, small_userDict


def computeMatrices(train_userDict,train_isbnDict,small_userDict,small_isbnDict, dict_row, dict_col):

    n = len(small_isbnDict)
    m = len(small_userDict)
    
    index = sorted(small_userDict.keys())
    columns = sorted(small_isbnDict.keys())

    dict_row = {k:v for v,k in enumerate(index)}
    dict_col = {k:v for v,k in enumerate(columns)}

    u = np.zeros((m,n)) 
    R = np.zeros((m,n))
    for user in train_userDict:
        for isbn in train_userDict[user]:
            try:
                u[dict_row[user]][dict_col[isbn]] = train_userDict[user][isbn]
                R[dict_row[user]][dict_col[isbn]] = 1
            except:
                continue

    for isbn in train_isbnDict:
        for user in train_isbnDict[isbn]:
            try:
                u[dict_row[user]][dict_col[isbn]] = train_isbnDict[isbn][user]
                R[dict_row[user]][dict_col[isbn]] = 1
            except:
                continue
                
    small_utility_DataFrame = pd.DataFrame(u, index = index, columns = columns)
    R = pd.DataFrame(R, index = index, columns = columns)

    return u, R, small_utility_DataFrame


def selectSample(i,b,list_user,small_userDict, small_isbnDict):

    test_index = [k for k in range((i*b),(i+1)*b)]
    train_index = [k for k in range(len(list_user)) if k not in test_index]

    train = list_user[train_index]
    test  = list_user[test_index]

    train_userDict = {}
    train_isbnDict = {isbn:{} for isbn in small_isbnDict}

    for user in train:

        train_userDict[user] = small_userDict[user]

    for isbn in small_isbnDict:
        for user in small_isbnDict[isbn]:

            try:
                train_userDict[user]
                train_isbnDict[isbn][user] = small_isbnDict[isbn][user]
            except:
                continue
                
                
    test_userDict = {}
    test_isbnDict = {isbn:{} for isbn in small_isbnDict}

    for user in test:

        test_userDict[user] = small_userDict[user]

    for isbn in small_isbnDict:
        for user in small_isbnDict[isbn]:

            try:
                test_userDict[user]
                test_isbnDict[isbn][user] = small_isbnDict[isbn][user]
            except:
                continue
                
    return train_userDict, train_isbnDict, test_userDict, test_isbnDict, test


#indices = [i for i in range(len(list_user))]
#indices = np.array(indices)
#k = 5
#b = len(small_userDict)//k


def selectSampleRandom(indices,b,list_user,small_userDict, small_isbnDict):
    
    test_index = np.random.choice(indices, size = b, replace=False)
    
    indices = np.delete(indices,test_index)
    
    train_index = [k for k in range(len(list_user)) if k not in test_index]
    

    train = list_user[train_index]
    test  = list_user[test_index]

    train_userDict = {}
    train_isbnDict = {isbn:{} for isbn in small_isbnDict}

    for user in train:

        train_userDict[user] = small_userDict[user]

    for isbn in small_isbnDict:
        for user in small_isbnDict[isbn]:

            try:
                train_userDict[user]
                train_isbnDict[isbn][user] = small_isbnDict[isbn][user]
            except:
                continue
                
    return train_userDict, train_isbnDict, test_userDict, test_isbnDict, test, indices

In [16]:
def mainItemItemsRMSE_clustering(utility_DataFrame,utility_DataFrame_clusters,R_clusters, new_userDict, new_isbnDict,
                                 index_user_cluster, index_book_cluster, score_min=0, k=3, zeros= "n"):
    

    rmse_dict = {j:{} for j in new_userDict.keys()}

    for user in new_userDict:

        for book in new_userDict[user]:

            if new_userDict[user][book] != "0":
                try:
                    new_isbnDict[book]
                    rmse_dict[user][book] = new_userDict[user][book]
                except:
                    continue
                    
            if new_userDict[user][book] == "0" and zeros != "n":  #if this is true score_min = -1
                try:
                    new_isbnDict[book]
                    rmse_dict[user][book] = "1"
                except:
                    continue
                    
    d = rmse_dict.keys()
    v = []

    for key in d:

        if rmse_dict[key] == {}:

            v.append(key)

    for i in v:

        del rmse_dict[i]

    rmse_vector_itemItems = []
    
    i = 0
    for user in rmse_dict:
        
        i = i+1
        if i%1000 == 0:
            
            print("Pause")
            time.sleep(30)

        for isbn in rmse_dict[user]:
            
            prediction_score = None
            user_number = user
            book_number = isbn
            
            
            try:
                
                prediction_score = CollaborativeFilteringItemItemsRMSE(utility_DataFrame, new_userDict, new_isbnDict, 
                                                                   user_number, score_min, book_number, k)
                true_score = utility_DataFrame[book_number][user_number]
                #print(true_score,prediction_score,"w")
            
            except:
                continue

            if prediction_score == None:
                
                user_cluster, book_cluster = convert(user_number,book_number, index_user_cluster, index_book_cluster)
                
                if R_clusters[book_cluster][user_cluster] == 1:
                
                    prediction_score = utility_DataFrame_clusters[book_cluster][user_cluster]
                    true_score = utility_DataFrame[book_number][user_number]

                    #print(true_score,prediction_score)
                    
            if prediction_score != None:

                rmse_vector_itemItems.append(tuple([user_number, book_number, prediction_score, true_score]))


    rmse = np.sqrt((np.array([u[2]-u[3] for u in rmse_vector_itemItems])**2).sum()/len(rmse_vector_itemItems))
    
    return rmse, rmse_vector_itemItems

In [ ]:
# with this approach I use the clusterings only when I don't have informations

In [17]:
small_isbnDict, small_userDict = createSampleDict(new_isbnDict, new_userDict, 20, 20)

In [9]:
u_clusters, R_clusters, utility_DataFrame_clusters = computeMatrices(CLUSTERS_USERS,CLUSTERS_ITEMS,
                                                                           CLUSTERS_USERS,CLUSTERS_ITEMS,
                                                                            clusters_dict_row,clusters_dict_col)

u, R, utility_DataFrame = computeMatrices(small_userDict,small_isbnDict,new_userDict,new_isbnDict,dict_row,dict_col)

In [19]:
rmse_test, rmse_vector_itemItems = mainItemItemsRMSE_clustering(utility_DataFrame,utility_DataFrame_clusters,R_clusters,
                                                                small_userDict,small_isbnDict,
                                                                index_user_cluster,index_book_cluster,
                                                                score_min=0,k=3)

In [12]:
rmse_vector_itemItems[:100]

[('190885', '1558743669', 8.0, 9.0),
 ('190885', '067088300X', 8.0, 6.0),
 ('190885', '1558531025', 8.3333333333333339, 7.0),
 ('190885', '0877733759', 7.0, 9.0),
 ('190885', '1558743316', 8.0, 8.0),
 ('190885', '0971880107', 8.0, 5.0),
 ('190885', '0804105820', 8.0, 8.0),
 ('225659', '0749395990', 8.0, 7.0),
 ('225659', '2070360024', 7.5, 8.0),
 ('225659', '0312305060', 7.5, 8.0),
 ('233711', '1558743669', 8.0, 7.0),
 ('233711', '0440211727', 7.666666666666667, 8.0),
 ('233711', '0385484518', 8.0, 9.0),
 ('233711', '0425144429', 7.666666666666667, 8.0),
 ('233711', '0446679593', 7.666666666666667, 10.0),
 ('233711', '0385335482', 8.0, 9.0),
 ('233711', '0316899984', 7.333333333333333, 5.0),
 ('233711', '0345384466', 9.0, 5.0),
 ('233711', '0446606189', 8.0, 7.0),
 ('233711', '0345439104', 7.333333333333333, 8.0),
 ('233711', '0425163385', 8.0, 7.0),
 ('233711', '0060096195', 8.6666666666666661, 9.0),
 ('233711', '0812570944', 7.0, 8.0),
 ('233711', '0452284295', 9.3333333333333339, 8.

# K-fold cross validation for collaborative filtering item-based

In [7]:
def itemItems_KfoldsCV_clustering(small_userDict,small_isbnDict, dict_row, dict_col, CLUSTERS_USERS,CLUSTERS_ITEMS,
                                                                            clusters_dict_row,clusters_dict_col,kk):

    RMSE = []
    #kk = 5
    list_user = np.array(sorted(small_userDict.keys()))
    b = len(small_userDict)//kk
    
    u_clusters, R_clusters, utility_DataFrame_clusters = computeMatrices(CLUSTERS_USERS,CLUSTERS_ITEMS,
                                                                           CLUSTERS_USERS,CLUSTERS_ITEMS,
                                                                            clusters_dict_row,clusters_dict_col)

    for iterator in range(kk):

        train_userDict, train_isbnDict, test_userDict, test_isbnDict, test =  selectSample(iterator,b,list_user,
                                                                                           small_userDict, small_isbnDict)
# use only train_userDict to create the utility matrix
        u, R, small_utility_DataFrame = computeMatrices(train_userDict,small_isbnDict,small_userDict,small_isbnDict, dict_row, dict_col)
# search users only in test_userDict
        rmse_test, rmse_vector_itemItems = mainItemItemsRMSE_clustering(small_utility_DataFrame,utility_DataFrame_clusters,R_clusters, 
                                                             test_userDict, test_isbnDict,index_user_cluster,index_book_cluster,
                                                             score_min=0, k=3)
        print("Ok")

        RMSE.append(rmse_test)
        
    return RMSE

In [8]:
small_userDict = {k:new_userDict[k] for k in list(new_userDict.keys())[:5000]}
small_isbnDict = {k:new_isbnDict[k] for k in list(new_isbnDict.keys())[:5000]}

In [20]:
small_isbnDict, small_userDict = createSampleDict(new_isbnDict, new_userDict, 20, 20)

In [21]:
RMSE = itemItems_KfoldsCV_clustering(small_userDict,small_isbnDict, dict_row, dict_col,CLUSTERS_USERS,CLUSTERS_ITEMS,
                                                                            clusters_dict_row,clusters_dict_col, 5)

Ok
Ok
Ok
Ok
Ok


In [22]:
print("rmse collaborative filtering item-based with clustering: ",np.mean(RMSE))

rmse collaborative filtering item-based with clustering:  1.6367878478
